<a href="https://colab.research.google.com/github/toinnn/Chat_Bot/blob/master/Chat_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
Git_Dir    = "/content/drive/MyDrive/Github_Dir/Chat_Bot"
Neural_Dir = "Machine-Learning__Deep-Learning"
with open("/content/drive/MyDrive/Github_Dir/acess_Token_Git.txt","r") as file:
    acess_Token_Git = file.read()
Git_Path   = "https://"+ acess_Token_Git + "@github.com/toinnn/" + Neural_Dir + ".git"
Git_CB_Path= "https://"+ acess_Token_Git + "@github.com/toinnn/" + "Chat_Bot" + ".git"

In [ ]:
#%cd "{Git_Dir}"
!git clone "{Git_Path}" ./temp
!git clone "{Git_CB_Path}" ./temp

!mv ./temp/* "{Git_Dir}"
!rm -rf ./temp 

!rsync -aP "{Git_Dir}"/*  ./
#!ln -s "/content/drive/MyDrive/Github_Dir/Chat_Bot" + Neural_Dir NLP

In [4]:
from BiLSTM import BiLSTM ,BiLSTM_Attention

print("Rodei 123")

teste
Rodei 123


In [ ]:
##!git add -u